In [1]:
from scipy import *
import numpy as np
import FNC

# Example 1.1.2

Recall the grade-school approximation to the number $\pi$.

$$
f(x) = \sin(x^2)
$$

In [2]:
p = 22/7
print(p)

3.142857142857143


Note that not all the digits displayed for `p` are the same as for $\pi$. As an approximation, its absolute and relative accuracy are

In [3]:
from numpy import pi
print("absolute accuracy: ",abs(p-pi))

absolute accuracy:  0.0012644892673496777


In [4]:
print("relative accuracy: ",abs(p-pi)/pi)
print("accurate digits: ",-np.log10(abs(p-pi)/pi))

relative accuracy:  0.0004024994347707008
accurate digits:  3.3952347251747166


# Example 1.1.3

There is no double precision number between $1$ and $1+\varepsilon_\text{mach}$. Thus the following difference is zero despite its appearance.

In [24]:
finfo = np.finfo(np.float32)

In [25]:
finfo.eps

1.1920929e-07

In [19]:
finfo.dtype

dtype('float64')

In [6]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

In [7]:
e = eps/2
e

1.1102230246251565e-16

In [9]:
1.0 + eps

1.0000000000000002

In [10]:
1.0 + eps/2 

1.0

In [12]:
(1.0 + eps/2) - 1.0 == eps/2

False

However, $1-\varepsilon_\text{mach}/2$ is a double precision number, so it and its negative are represented exactly:

In [14]:
1.0 + (eps/2 - 1.0) == eps/2

True

This is now the "correct" result. But we have found a rather shocking breakdown of the associative law of addition!

# Example 1.3.2

Here we show how to use `horner` to evaluate a polynomial. We first define a vector of the coefficients of $p(x)=(x−1)^3=x^3−3x^2+3x−1$, in descending degree order. Note that the textbook's functions are all in a namespace called `FNC`, to help distinguish them from other Python commands and modules.


In [3]:
c = np.array([1,-3,3,-1])
print( FNC.horner(c,1.6) )

0.21600000000000041


In [4]:
(1.6 - 1)**3

0.2160000000000001

The above is the value of $p(1.6)$, up to a rounding error.

# Example 1.3.3

Our first step is to construct a polynomial with six known roots.

In [6]:
help(np.poly)

Help on function poly in module numpy:

poly(seq_of_zeros)
    Find the coefficients of a polynomial with the given sequence of roots.
    
    .. note::
       This forms part of the old polynomial API. Since version 1.4, the
       new polynomial API defined in `numpy.polynomial` is preferred.
       A summary of the differences can be found in the
       :doc:`transition guide </reference/routines.polynomials>`.
    
    Returns the coefficients of the polynomial whose leading coefficient
    is one for the given sequence of zeros (multiple roots must be included
    in the sequence as many times as their multiplicity; see Examples).
    A square matrix (or array, which will be treated as a matrix) can also
    be given, in which case the coefficients of the characteristic polynomial
    of the matrix are returned.
    
    Parameters
    ----------
    seq_of_zeros : array_like, shape (N,) or (N, N)
        A sequence of polynomial roots, or a square array or matrix object.
    
  

In [21]:
r = np.array([-2.0,-1,1,1,3,6])
p = np.poly(r)
print(p)

[  1.  -8.   6.  44. -43. -36.  36.]


Now we use a standard numerical method for finding those roots, pretending that we don't know them already.

In [22]:
px = np.poly1d(p)
px

poly1d([  1.,  -8.,   6.,  44., -43., -36.,  36.])

In [23]:
px.roots

array([ 6.        ,  3.        , -2.        , -1.        ,  0.99999999,
        1.00000001])

In [24]:
r_computed = np.sort(px.roots)
print(r_computed)

[-2.         -1.          0.99999999  1.00000001  3.          6.        ]


Here are the relative errors in each of the computed roots.

In [25]:
print(abs(r - r_computed) / abs(r))

[3.33066907e-16 2.22044605e-16 1.23423448e-08 1.23423434e-08
 1.18423789e-15 1.18423789e-15]


It seems that the forward error is acceptably close to machine epsilon for double precision in all cases except the double root at $x=1$. This is not a surprise, though, given the poor conditioning at such roots.

Let's consider the backward error. The data in the rootfinding problem are the polynomial coefficients. We can apply poly to find the coefficients of the polynomial (that is, the data) whose roots were actually computed by the numerical algorithm.

In [26]:
p_computed = np.poly(r_computed)
print(p_computed)

[  1.  -8.   6.  44. -43. -36.  36.]


In [29]:
p_computed[1] - p[1]

4.440892098500626e-15

In [30]:
p_computed[2] - p[2]

-1.7763568394002505e-15

We find that in a relative sense, these coefficients are very close to those of the original, exact polynomial:

In [31]:
print(abs(p-p_computed)/abs(p))

[0.00000000e+00 5.55111512e-16 2.96059473e-16 4.84460956e-16
 1.65242497e-15 1.97372982e-16 1.57898386e-15]


In summary, even though there are some computed roots relatively far from their correct values, they are nevertheless the roots of a polynomial that is very close to the original.

# Example 1.3.4

In [14]:
a = 1.0
b = -(1e6+1e-6)
c = 1.0

In [15]:
x1 = (-b + np.sqrt(b*b-4*a*c)) / (2*a)
print(x1)

1000000.0


So far, so good. But:

In [16]:
x2 = (-b - np.sqrt(b*b-4*a*c)) / (2*a)
print(x2)

1.00000761449337e-06


In [17]:
a2 = 1.0
b2 = -(x1+x2)
c2 = x1*x2

In [18]:
a2, b2, c2

(1.0, -1000000.000001, 1.00000761449337)

In [19]:
a, b, c

(1.0, -1000000.000001, 1.0)

In [20]:
np.sqrt((a-a2)**2 + (b-b2)**2 + (c-c2)**2)

7.614493370056152e-06

The first value is correct to all stored digits, but the second has fewer than six accurate digits:

In [5]:
print( -np.log10(abs(1e-6-x2)/1e-6 ) )

5.118358987126217


In [6]:
-b

1000000.000001

In [7]:
np.sqrt(b*b - 4*a*c)

999999.999999

In [8]:
-b - np.sqrt(b*b-4*a*c)

2.00001522898674e-06

# Example 1.3.5

In [9]:
a = 1.0
b = -(1e6+1e-6)
c = 1.0

First, we find the "good" root using the quadratic forumla. 

In [11]:
x1 = (-b - np.sign(b)*np.sqrt(b*b-4*a*c)) / (2*a)
print(x1)

1000000.0


Then we use the alternative formula for computing the other root. 

In [13]:
x2 = c/(a*x1)
print(x2)
print(x2 - 1e-6)

1e-06
0.0


---

## Integral example

$$
y_n = \int_0^1 \frac{x^n}{x+10} dx, \qquad n = 0, 1, \ldots, 30.
$$

Algorithm:

$$
y_n = \frac1n - 10 y_{n-1}, \qquad y_0 = \ln 11 - \ln 10.
$$

In [52]:
ytrue = [
    np.log(11/10),
    1 - 10*np.log(11/10),
    -9.5 + 100*np.log(11/10),
    1/3 + 95 - 1000*np.log(11/10)
]
ytrue

[0.09531017980432493,
 0.04689820195675065,
 0.03101798043249282,
 0.023153529008396845]

In [54]:
np.abs(ytrue - y[0:4]) / np.abs(ytrue)

array([0.00000000e+00, 0.00000000e+00, 2.14757313e-14, 6.95282082e-14])

In [47]:
y = np.zeros(31)
y[0] = np.log(11/10)
print([y[0]])
for n in range(1,31):
    y[n] = 1/n - 10*y[n-1]
    print([1/n, 10*y[n-1], y[n]])
y

[0.09531017980432493]
[1.0, 0.9531017980432493, 0.04689820195675065]
[0.5, 0.4689820195675065, 0.031017980432493486]
[0.3333333333333333, 0.31017980432493486, 0.023153529008398455]
[0.25, 0.23153529008398455, 0.018464709916015454]
[0.2, 0.18464709916015454, 0.015352900839845474]
[0.16666666666666666, 0.15352900839845474, 0.013137658268211921]
[0.14285714285714285, 0.13137658268211921, 0.011480560175023635]
[0.125, 0.11480560175023635, 0.010194398249763648]
[0.1111111111111111, 0.10194398249763648, 0.009167128613474629]
[0.1, 0.09167128613474629, 0.008328713865253717]
[0.09090909090909091, 0.08328713865253717, 0.007621952256553738]
[0.08333333333333333, 0.07621952256553738, 0.00711381076779595]
[0.07692307692307693, 0.0711381076779595, 0.005784969245117427]
[0.07142857142857142, 0.05784969245117427, 0.013578878977397152]
[0.06666666666666667, 0.13578878977397152, -0.06912212310730485]
[0.0625, -0.6912212310730486, 0.7537212310730486]
[0.058823529411764705, 7.537212310730485, -7.47838878

array([ 9.53101798e-02,  4.68982020e-02,  3.10179804e-02,  2.31535290e-02,
        1.84647099e-02,  1.53529008e-02,  1.31376583e-02,  1.14805602e-02,
        1.01943982e-02,  9.16712861e-03,  8.32871387e-03,  7.62195226e-03,
        7.11381077e-03,  5.78496925e-03,  1.35788790e-02, -6.91221231e-02,
        7.53721231e-01, -7.47838878e+00,  7.48394434e+01, -7.48341802e+02,
        7.48346802e+03, -7.48346326e+04,  7.48346371e+05, -7.48346367e+06,
        7.48346367e+07, -7.48346367e+08,  7.48346367e+09, -7.48346367e+10,
        7.48346367e+11, -7.48346367e+12,  7.48346367e+13])

In [55]:
import scipy.integrate as integrate

In [77]:
def f(x,n):
    return x**n/(x+10)

yquad = [integrate.quad(lambda x: f(x,n), 0, 1) for n in range(31)]
yquad

[(0.09531017980432485, 1.0581555609992505e-15),
 (0.0468982019567514, 5.206746362590598e-16),
 (0.031017980432486002, 3.443687605351853e-16),
 (0.02315352900847329, 2.570558100653352e-16),
 (0.018464709915267108, 2.0499946090953966e-16),
 (0.015352900847328937, 1.704514401549166e-16),
 (0.013137658193377286, 1.45857306159428e-16),
 (0.011480560923370004, 1.2745983072737226e-16),
 (0.010194390766299978, 1.1318047350772327e-16),
 (0.009167203448111371, 1.0177640339516371e-16),
 (0.008327965518886312, 9.245899067351973e-17),
 (0.007629435720227788, 8.470375201494505e-17),
 (0.00703897613105546, 7.814833370484675e-17),
 (0.006533315612522328, 7.253437420165296e-17),
 (0.006095415303348168, 6.767270414429668e-17),
 (0.005712513633184993, 6.342164164047084e-17),
 (0.005374863668150088, 5.967297398601455e-17),
 (0.0050748927302638434, 5.634262756641743e-17),
 (0.004806628252917125, 5.3364293572023825e-17),
 (0.004565296418197191, 5.068497197721279e-17),
 (0.00434703581802811, 3.82190418601408

In [82]:
ygauss = [integrate.quadrature(lambda x: f(x,n), 0, 1, tol=1e-15, rtol=1e-15, maxiter=1000) for n in range(31)]
ygauss

[(0.09531017980432485, 1.3877787807814457e-17),
 (0.046898201956751394, 9.020562075079397e-17),
 (0.031017980432486, 8.36136715420821e-16),
 (0.023153529008473294, 3.469446951953614e-18),
 (0.018464709915267115, 6.245004513516506e-17),
 (0.015352900847328946, 4.683753385137379e-16),
 (0.013137658193377292, 1.734723475976807e-18),
 (0.011480560923370006, 3.122502256758253e-17),
 (0.010194390766299978, 2.706168622523819e-16),
 (0.009167203448111406, 3.2959746043559335e-17),
 (0.00832796551888635, 5.0306980803327406e-17),
 (0.007629435720227826, 1.196959198423997e-16),
 (0.007038976131055433, 6.331740687315346e-17),
 (0.0065333156125223, 7.19910242530375e-17),
 (0.006095415303348141, 2.688821387764051e-17),
 (0.005712513633184965, 9.532305500492555e-16),
 (0.005374863668150111, 5.637851296924623e-17),
 (0.005074892730263868, 0.0),
 (0.004806628252917148, 5.568462357885551e-16),
 (0.004565296418197149, 6.765421556309548e-17),
 (0.004347035818028068, 3.729655473350135e-17),
 (0.004148689438

In [74]:
help(integrate.quadrature)

Help on function quadrature in module scipy.integrate._quadrature:

quadrature(func, a, b, args=(), tol=1.49e-08, rtol=1.49e-08, maxiter=50, vec_func=True, miniter=1)
    Compute a definite integral using fixed-tolerance Gaussian quadrature.
    
    Integrate `func` from `a` to `b` using Gaussian quadrature
    with absolute tolerance `tol`.
    
    Parameters
    ----------
    func : function
        A Python function or method to integrate.
    a : float
        Lower limit of integration.
    b : float
        Upper limit of integration.
    args : tuple, optional
        Extra arguments to pass to function.
    tol, rtol : float, optional
        Iteration stops when error between last two iterates is less than
        `tol` OR the relative change is less than `rtol`.
    maxiter : int, optional
        Maximum order of Gaussian quadrature.
    vec_func : bool, optional
        True or False if func handles arrays as arguments (is
        a "vector" function). Default is True.
 